In [2]:
"""
verificar_tamanho_sentencas.py
Verifica distribuição de tamanho das sentenças
"""

import pandas as pd
import re

print("="*60)
print("ANÁLISE: TAMANHO DAS SENTENÇAS")
print("="*60 + "\n")

df = pd.read_csv('processos_tjac_com_genero.csv')
df_sentencas = df[df['procedente'].notna()].copy()

# Calcular tamanhos
df_sentencas['num_caracteres'] = df_sentencas['desc'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)
df_sentencas['num_palavras'] = df_sentencas['desc'].apply(lambda x: len(str(x).split()) if pd.notna(x) else 0)

# Estimar tokens (aproximadamente 1.3 palavras = 1 token em português)
df_sentencas['tokens_estimados'] = (df_sentencas['num_palavras'] * 1.3).astype(int)

print("Estatísticas de tamanho:\n")
print(f"Caracteres:")
print(f"  Média: {df_sentencas['num_caracteres'].mean():.0f}")
print(f"  Mediana: {df_sentencas['num_caracteres'].median():.0f}")
print(f"  Máximo: {df_sentencas['num_caracteres'].max():.0f}")

print(f"\nPalavras:")
print(f"  Média: {df_sentencas['num_palavras'].mean():.0f}")
print(f"  Mediana: {df_sentencas['num_palavras'].median():.0f}")
print(f"  Máximo: {df_sentencas['num_palavras'].max():.0f}")

print(f"\nTokens estimados:")
print(f"  Média: {df_sentencas['tokens_estimados'].mean():.0f}")
print(f"  Mediana: {df_sentencas['tokens_estimados'].median():.0f}")
print(f"  Máximo: {df_sentencas['tokens_estimados'].max():.0f}")

# Distribuição
print("\n" + "="*60)
print("DISTRIBUIÇÃO POR FAIXA DE TOKENS")
print("="*60 + "\n")

faixas = [
    (0, 512, 'Bert 512'),
    (513, 1024, '513 - 1024'),
    (1025, 4096, '1025 - 4096'),
    (4097, 16384, '> 4096')
]

for min_tok, max_tok, modelo in faixas:
    count = ((df_sentencas['tokens_estimados'] > min_tok) & (df_sentencas['tokens_estimados'] <= max_tok)).sum()
    pct = count / len(df_sentencas) * 100
    print(f"{modelo:25} ({min_tok:5}-{max_tok:5} tokens): {count:3} ({pct:5.1f}%)")

ANÁLISE: TAMANHO DAS SENTENÇAS

Estatísticas de tamanho:

Caracteres:
  Média: 1243
  Mediana: 866
  Máximo: 13363

Palavras:
  Média: 198
  Mediana: 140
  Máximo: 2112

Tokens estimados:
  Média: 257
  Mediana: 181
  Máximo: 2745

DISTRIBUIÇÃO POR FAIXA DE TOKENS

Bert 512                  (    0-  512 tokens): 682 ( 90.2%)
513 - 1024                (  513- 1024 tokens):  48 (  6.3%)
1025 - 4096               ( 1025- 4096 tokens):  25 (  3.3%)
> 4096                    ( 4097-16384 tokens):   0 (  0.0%)


In [9]:
"""
gerar_csv_xlsx_nlp_final.py
Gera CSV e XLSX com análise NLP completa
Sem estatísticas (será feito em arquivo separado)
"""

from transformers import pipeline
import pandas as pd
import textstat

print("="*80)
print("GERAÇÃO DE ARQUIVOS FINAIS - ANÁLISE NLP")
print("="*80 + "\n")

# Configurar textstat para português
textstat.set_lang('pt')

# ==================== CARREGAR DATASET ====================
print("1. Carregando dataset...\n")

df = pd.read_csv('processos_tjac_com_genero.csv')
df_analise = df[df['procedente'].notna()].copy()

print(f"Dataset carregado: {len(df)} processos")
print(f"Com sentença identificada: {len(df_analise)} processos\n")

# ==================== CARREGAR MODELO BERT-PT ====================
print("2. Carregando modelo BERT-PT...\n")

analisador_sentimento = pipeline(
    "sentiment-analysis",
    model="lipaoMai/bert-sentiment-model-portuguese",
    framework='pt', #força o uso do torch ao invés do keras
    device=-1, ##para rodar na GPU altere para 0 (zero). 1070TI não teve compatibilidade com a versão do torch, por isso ficou na cpu
    truncation=True,
    max_length=512
)

print("BERT-PT carregado!\n")

# ==================== MAPEAMENTOS PARA PORTUGUÊS ====================

MAP_SENTIMENTO = {
    'POSITIVE': 'positivo',
    'NEGATIVE': 'negativo',
    'NEUTRAL': 'neutro'
}

def classificar_flesch_ajustado(score):
    """Classifica complexidade adaptado para textos jurídicos"""
    score = max(0, score)
    
    if score >= 56:
        return 'baixa'
    elif score >= 36:
        return 'média'
    else:
        return 'alta'

def analisar_sentenca(texto):
    """Analisa sentimento e complexidade"""
    if pd.isna(texto) or not texto:
        return None, 0.0, None, 0.0
    
    texto_str = str(texto)
    
    # SENTIMENTO
    try:
        sent_result = analisador_sentimento(texto_str)[0]
        sentimento_en = sent_result['label']
        sentimento_pt = MAP_SENTIMENTO.get(sentimento_en, sentimento_en.lower())
        sent_score = sent_result['score']
    except:
        sentimento_pt = None
        sent_score = 0.0
    
    # COMPLEXIDADE
    try:
        flesch_raw = textstat.flesch_reading_ease(texto_str)
        flesch_score = max(0, flesch_raw)
        complexidade = classificar_flesch_ajustado(flesch_score)
    except:
        flesch_score = 0.0
        complexidade = None
    
    return sentimento_pt, sent_score, complexidade, flesch_score

# ==================== PROCESSAR TODAS AS SENTENÇAS ====================
print("3. Processando 756 sentenças com NLP...\n")
sentimentos = []
sentimentos_scores = []
complexidades = []
complexidades_scores = []

for i, desc in enumerate(df_analise['desc'], 1):
    sentimento, sent_score, complexidade, flesch_score = analisar_sentenca(desc)
    
    sentimentos.append(sentimento)
    sentimentos_scores.append(sent_score)
    complexidades.append(complexidade)
    complexidades_scores.append(flesch_score)
    
    if i % 50 == 0:
        print(f"   Processadas: {i}/{len(df_analise)}")

# Adicionar colunas
df_analise['sentimento_sentenca'] = sentimentos
df_analise['sentimento_score'] = sentimentos_scores
df_analise['complexidade_sentenca'] = complexidades
df_analise['complexidade_score'] = complexidades_scores

print(f"\n{len(df_analise)} sentenças processadas!\n")

# ==================== CRIAR LEGENDA ====================
print("4. Criando arquivo de legenda...\n")

legenda = """
LEGENDA - ANÁLISE NLP DAS SENTENÇAS
====================================

COLUNAS ADICIONADAS:
-------------------

1. sentimento_sentenca
   Classificação do sentimento/tom do texto da sentença
   Valores possíveis:
   - positivo: Tom favorável/ameno
   - neutro: Tom técnico/formal (maioria dos casos jurídicos)
   - negativo: Tom severo/desfavorável
   
   Método: Modelo BERT-PT fine-tunado para português
   Modelo: lipaoMai/bert-sentiment-model-portuguese

2. sentimento_score
   Confiança do modelo na classificação do sentimento
   Valores: 0.0 a 1.0 (0% a 100%)
   Exemplo: 0.65 = 65% de confiança
   
   Interpretação:
   - 0.0-0.5: Baixa confiança
   - 0.5-0.7: Confiança média
   - 0.7-1.0: Alta confiança

3. complexidade_sentenca
   Nível de complexidade/dificuldade de leitura do texto
   Valores possíveis:
   - alta: Texto muito técnico/complexo (70.5% dos casos)
   - média: Complexidade padrão jurídica (29.4% dos casos)
   - baixa: Texto relativamente simples (0.1% dos casos)
   
   Método: Índice Flesch Reading Ease adaptado para português
   Referência acadêmica: Flesch, R. (1948)

4. complexidade_score
   Score numérico do Índice Flesch Reading Ease
   Valores: 0 a 100 (quanto MAIOR, mais FÁCIL de ler)
   
   Interpretação:
   - 0-30: Muito difícil (nível universitário/técnico)
   - 30-50: Difícil (nível superior)
   - 50-60: Moderadamente difícil
   - 60-70: Razoável
   - 70-100: Fácil
   
   Média dos textos jurídicos: ~22.8 (muito difícil)
   
   Classificação automática:
   - alta complexidade: score 0-35
   - média complexidade: score 36-55
   - baixa complexidade: score 56-100

METODOLOGIA:
-----------
Sentimento: Modelo de aprendizado profundo (BERT) treinado em textos portugueses
Complexidade: Métrica linguística consolidada (Flesch Reading Ease)

OBSERVAÇÕES:
-----------
- Textos jurídicos são naturalmente formais (maioria "neutro")
- Textos jurídicos são naturalmente complexos (maioria "alta")
- Scores de confiança < 0.6 devem ser interpretados com cautela
- Análise determinística: mesmo texto = mesmo resultado
"""

with open('LEGENDA_NLP.txt', 'w', encoding='utf-8') as f:
    f.write(legenda)

print("Legenda salva: LEGENDA_NLP.txt\n")

# ==================== SALVAR ARQUIVOS ====================
print("5. Salvando arquivos finais...\n")

# CSV com encoding UTF-8-SIG (compatível com Excel)
df_analise.to_csv('processos_tjac_completo_nlp.csv', 
                  index=False, 
                  encoding='utf-8-sig')
print("Salvo: processos_tjac_completo_nlp.csv")

# CSV com separador ponto-e-vírgula (Excel brasileiro)
df_analise.to_csv('processos_tjac_completo_nlp_excel.csv', 
                  index=False, 
                  encoding='utf-8-sig',
                  sep=';')
print("Salvo: processos_tjac_completo_nlp_excel.csv (sep=;)")

# Excel nativo
df_analise.to_excel('processos_tjac_completo_nlp.xlsx', 
                    index=False, 
                    engine='openpyxl')
print("Salvo: processos_tjac_completo_nlp.xlsx")

# ==================== RESUMO ====================
print("\n" + "="*80)
print("ARQUIVOS GERADOS COM SUCESSO!")
print("="*80)
print(f"\nTotal de processos: {len(df_analise)}")
print(f"Total de colunas: {len(df_analise.columns)}")
print()
print("Arquivos gerados:")
print("  1. processos_tjac_completo_nlp.csv (padrão)")
print("  2. processos_tjac_completo_nlp_excel.csv (Excel BR)")
print("  3. processos_tjac_completo_nlp.xlsx (Excel nativo)")
print("  4. LEGENDA_NLP.txt (documentação)")
print()
print("Novas colunas NLP:")
print("  • sentimento_sentenca (positivo/neutro/negativo)")
print("  • sentimento_score (0-1)")
print("  • complexidade_sentenca (alta/média/baixa)")
print("  • complexidade_score (0-100)")
print("="*80)

GERAÇÃO DE ARQUIVOS FINAIS - ANÁLISE NLP

1. Carregando dataset...

Dataset carregado: 1200 processos
Com sentença identificada: 756 processos

2. Carregando modelo BERT-PT...



Device set to use cpu


BERT-PT carregado!

3. Processando 756 sentenças com NLP...

   Processadas: 50/756
   Processadas: 100/756
   Processadas: 150/756
   Processadas: 200/756
   Processadas: 250/756
   Processadas: 300/756
   Processadas: 350/756
   Processadas: 400/756
   Processadas: 450/756
   Processadas: 500/756
   Processadas: 550/756
   Processadas: 600/756
   Processadas: 650/756
   Processadas: 700/756
   Processadas: 750/756

756 sentenças processadas!

4. Criando arquivo de legenda...

Legenda salva: LEGENDA_NLP.txt

5. Salvando arquivos finais...

Salvo: processos_tjac_completo_nlp.csv
Salvo: processos_tjac_completo_nlp_excel.csv (sep=;)
Salvo: processos_tjac_completo_nlp.xlsx

ARQUIVOS GERADOS COM SUCESSO!

Total de processos: 756
Total de colunas: 20

Arquivos gerados:
  1. processos_tjac_completo_nlp.csv (padrão)
  2. processos_tjac_completo_nlp_excel.csv (Excel BR)
  3. processos_tjac_completo_nlp.xlsx (Excel nativo)
  4. LEGENDA_NLP.txt (documentação)

Novas colunas NLP:
  • sentimento_s